## Step(1): Create Tools

In [1]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
from langchain.tools import tool
import requests



### Tool(1)  for Getting Conversion Factor

In [2]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/Enter Your API/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

### Tool(2) for Conversion

In [3]:
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [4]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [5]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1746489601,
 'time_last_update_utc': 'Tue, 06 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1746576001,
 'time_next_update_utc': 'Wed, 07 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 84.3508}

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':84.5939})

845.9390000000001

## Step(2) Tool Binding

In [7]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.7 MB/s eta 0:00:00


In [8]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

In [9]:
import os
os.environ["GROQ_API_KEY"] = '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'

In [10]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [11]:
llm = ChatGroq(model_name="llama3-8b-8192", api_key=groq_api_key)

In [15]:
llm.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.001812712, 'queue_time': 0.021421167999999997, 'total_time': 0.022646045}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bb31eae-80bf-4196-93c0-d56f030869e5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [16]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [17]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 USD to INR')]

In [18]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [19]:
ai_message = llm_with_tools.invoke(messages)

In [20]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3bbv', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_fzc9', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":{"name":"some_function","result":""}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 1070, 'total_tokens': 1234, 'completion_time': 0.136666667, 'prompt_time': 0.133937379, 'queue_time': 0.023182370000000008, 'total_time': 0.270604046}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b86992-9fec-47a1-8389-90a33aba50c5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_3bbv', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_f

In [21]:
messages.append(ai_message)

In [22]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_3bbv',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10,
   'conversion_factor': {'name': 'some_function', 'result': ''}},
  'id': 'call_fzc9',
  'type': 'tool_call'}]

In [23]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [24]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3bbv', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_fzc9', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":{"name":"some_function","result":""}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 1070, 'total_tokens': 1234, 'completion_time': 0.136666667, 'prompt_time': 0.133937379, 'queue_time': 0.023182370000000008, 'total_time': 0.270604046}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-25b86992-9fec-47a1-8389-90a33aba50c5-0', tool_calls=[{'name': 'get_conversion_factor', 

In [25]:
llm_with_tools.invoke(messages).content

'84.35'

In [26]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

<ipython-input-26-23d1b63d4a11>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [27]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm just a language model, I'm doing well, thank you for asking!"
}
```


> Finished chain.
